In [5]:
# Generate demand for simulation analysis

In [44]:
import itertools
import numpy as np
from scipy.stats import norm
from scipy.stats import poisson
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [71]:
# Consider two rate classes for simplicity
n_class = 2
los = 3
capacity = 50
# Consider two demand scenario, by changing capacity
demand_intensity = np.array([0.9, 1.5])
# Slope for linear demand curve
slope = np.array([-0.1, -0.15])
intercept = np.array([21, 19])
# Nightly rates for Monday stay night through Sunday stay night
# Higher weekday rates imply typical hotel settings and higher weekend rates imply resorts
init_rate1 = np.array([135, 135, 135, 135, 135, 108, 108])
init_rate2 = np.array([115, 115, 115, 115, 115, 92, 92])

In [102]:
# Take the slope of -0.1 for rate class 1, then the mean true demand point for any arrival date/lengthof-
# stay combination is passed through by the line defined by this slope; 30 points on the price 
# axis for each arrival date/length-of-stay combination over the first 21 simulation days 
# are randomly generated from the interval (init_rate-50, init_rate+50)
# Sunday-one night stay as an example
drawsize = 30
# Random draw for one night stay for each day of week
random_rates1_one = [np.random.randint(init_rate1[i]-50, init_rate1[i]+50+1, drawsize) 
                     for i in range(7)]
random_rates2_one = [np.random.randint(init_rate2[i]-50, init_rate2[i]+50+1, drawsize) 
                     for i in range(7)]

# Construct rates for more than one night stay, up to 3-night stay
# Sunday night arrival, two night stay rates equal sunday one night + monday one night stay
random_rates1_two = [random_rates1_one[i%7] 
                     + random_rates1_one[(i+1)%7] 
                     for i in range(7)]

random_rates2_two = [random_rates2_one[i%7] 
                     + random_rates2_one[(i+1)%7] 
                     for i in range(7)]

random_rates1_three = [random_rates1_one[i%7] 
                       + random_rates1_one[(i+1)%7] 
                       + random_rates1_one[(i+2)%7] 
                       for i in range(7)]

random_rates2_three = [random_rates2_one[i%7] 
                       + random_rates2_one[(i+1)%7] 
                       + random_rates2_one[(i+2)%7] 
                       for i in range(7)]

In [122]:
# Change type to numpy array
random_rates1_one = np.array(random_rates1_one).reshape(7, drawsize)
random_rates1_two = np.array(random_rates1_two).reshape(7, drawsize)
random_rates1_three = np.array(random_rates1_three).reshape(7, drawsize)
random_rates1 = [(random_rates1_one[i], random_rates1_two[i], random_rates1_three[i]) 
                 for i in range(7)]

In [124]:
# Reshape array, where axis 0 = day of week, axis 1 = length of stay, and axis 3 = random draw
random_rates1 = np.array(random_rates1).reshape(7, 3, drawsize)

In [125]:
random_rates1

array([[[178, 173,  93, 125,  93, 167,  87, 146, 159,  99, 183, 153,
          98,  91, 137, 105, 121, 112, 159, 155, 170, 138, 112, 119,
         178, 125, 155,  99, 125, 131],
        [265, 287, 208, 232, 276, 271, 188, 285, 303, 222, 323, 272,
         252, 207, 283, 260, 269, 245, 309, 240, 332, 285, 254, 271,
         357, 287, 269, 211, 244, 312],
        [354, 406, 357, 402, 403, 431, 284, 447, 420, 337, 411, 416,
         373, 335, 397, 402, 358, 410, 417, 361, 465, 422, 365, 400,
         515, 420, 445, 318, 339, 433]],

       [[ 87, 114, 115, 107, 183, 104, 101, 139, 144, 123, 140, 119,
         154, 116, 146, 155, 148, 133, 150,  85, 162, 147, 142, 152,
         179, 162, 114, 112, 119, 181],
        [176, 233, 264, 277, 310, 264, 197, 301, 261, 238, 228, 263,
         275, 244, 260, 297, 237, 298, 258, 206, 295, 284, 253, 281,
         337, 295, 290, 219, 214, 302],
        [274, 393, 400, 408, 424, 415, 340, 480, 433, 417, 322, 430,
         380, 423, 353, 436, 345, 457, 

In [69]:
# Reshape by setting axis 0 = day of week, axis 1 = rate class, and axis 2 = 30 random rate draw
random_rates_single = np.array(random_rates_single).reshape(7, n_class, drawsize)
# Check accuracy, result should be similar to init_rate
np.mean(random_rates, axis = 2)

array([[135.46666667, 108.53333333],
       [133.86666667, 108.56666667],
       [133.43333333, 117.63333333],
       [130.5       , 108.6       ],
       [134.83333333, 120.        ],
       [105.7       ,  95.5       ],
       [ 97.3       ,  90.03333333]])

In [ ]:
# Construct rates for more than one night stay, up to 3-night stay


In [34]:
# 30 corresponding points on the demand axis are generated via a draw from
# the Poisson distribution with a mean equal to that predicted by the slope and y-intercept;

sunday1_demand_mean = intercept[0] + slope[0]*sunday1_rate
sunday1_demand = [poisson.rvs(mu, size = 1) for mu in sunday1_demand_mean]
sunday1_demand = np.array(sunday1_demand).reshape(len(sunday1_rate))

In [35]:
sunday1_demand

array([ 8,  7,  1, 10, 10,  5,  9,  0,  5,  5,  1, 10,  3,  7,  9,  2,  7,
        8,  0, 13,  5,  9,  4,  9,  7,  2, 11, 14,  8,  4])